In [ ]:
import requests, time
from bs4 import BeautifulSoup
from datetime import timedelta, datetime as dt
import pandas as pd, re
import numpy as np
from IPython.display import display,clear_output
from ipywidgets import IntProgress
import itertools as it
from geopy.geocoders import Nominatim
# from selenium.webdriver.common.by import By
from tqdm.notebook import tqdm_notebook
import nltk
from nltk import FreqDist
import matplotlib.pyplot as plt
from nltk.corpus import stopwords, words
from wordcloud import WordCloud, ImageColorGenerator
try:
  from uszipcode import SearchEngine
except:
  !pip install uszipcode
  !pip install fuzzywuzzy
  from uszipcode import SearchEngine

from fuzzywuzzy import process, fuzz
search = SearchEngine(simple_zipcode=True)
tqdm_notebook.pandas(position=0, leave=True)
geolocator = Nominatim(user_agent="geocoding_jobscraper")
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [ ]:
roles = ['business analyst','data scientist','data analyst','marketing analyst','product analyst','strategy analyst','decision scientist','analyst']

In [ ]:
skillz = {
  #  'excel':'Excel',
   'python':'Python',
  #  'r':'R',
   'sql':'SQL',
  #  'c':'C',
   'java':'Java',
   'c++':'C++',
   'spark': "Spark",
   'hadoop': 'Hadoop',
   'aws': 'AWS',
   'sas': 'SAS',
   'matlab': "Matlab",
   'scala': 'Scala',
   'tableau': 'Tableau',
   'hive': 'Hive',
   'tensorflow': 'Tensorflow',
   'javascript': 'Javascript',
   'perl': 'Perl',
   'linux': 'Linux',
   'unix': 'Unix',
   'mapreduce': 'Mapreduce',
   'scikit': 'Scikit',
   'html': 'HTML',
   'oracle': 'Oracle',
   'mongodb': 'Mongodb'}
majors = {
   'computer science': 'Computer Science',
   'statistics': 'Statistics',
   'mathematics': 'Mathematics',
   'economics': 'Economics',
   'biology': 'Biology',
   'quantitative': 'Quantitative',
   'data science': 'Data Science',
   'analytics': 'Analytics',
   'chemical engineer': 'Chemical Engineer'
}
degrees = {
   'high school':'High School',
   'bachelor': 'Bachelor',
   'master': 'Master',
   'phd': 'PhD'
}

In [ ]:
def check(item):
    if item == None:
        return None
    else:
        return item.text.strip()

def geocode(x):
    if x==None:
        return 0,0
    try:
        location = geolocator.geocode(x)
        return location.latitude, location.longitude
    except Exception as e:
        return 0,0
    
'''Example of geocode extraction'''
# location = geolocator.geocode("175 5th Avenue NYC")
# print(location.address)
# print((location.latitude, location.longitude))
# print(location.raw)
geocode('Phoenix, AZ, US')

### Indeed.com

In [ ]:
def build_dataset(seed_urls):
    news_data = []
    for url in tqdm_notebook(seed_urls):
        try:
            news_category = url.split('/')[-1]
            data = requests.get(url)
            soup = BeautifulSoup(data.content, 'html.parser')

            news_articles = [{'JobTitle': title.a['title'],
                              'Company': company.text.strip(),
                              'Location': location.text.strip(),
                              'DatePosted':date.text.strip(),
                             'Link':'https://www.indeed.com'+title.a['href']}

                                for title, company, location, date in 
                                 zip(soup.find_all('h2', 
                                                   class_=["title"]),
                                     soup.find_all('span', 
                                                   class_=["company"]),
                                    soup.find_all('div', 
                                                   class_=["location accessible-contrast-color-location"]),
                                     soup.find_all('span',class_='date'))
                            ]
            news_data.extend(news_articles)
        except:
            pass
#         print(soup)
#         soup.find_all('div',attrs={"class":"title"})
#         for headline, article in zip(soup.find_all('div',class_=["title"]),soup.find_all('span', class_=["company"])):
#             print(headline.a['title'], article.text.strip())
        
    df =  pd.DataFrame(news_data)
    df = df[['JobTitle', 'Company', 'Location','DatePosted','Link']]
    return df

In [ ]:
indeed_data = pd.DataFrame()
for role in roles:
  base_url = 'https://www.indeed.com/jobs?q='+role.replace(' ','+')+'&l=USA&sort=date&start='

  seed_urls = ['https://www.indeed.com/q-'+role.replace(' ','-')+'-l-USA-jobs.html']+[base_url+str((i+1)*10) for i in range(400)]

  news_df = build_dataset(seed_urls)
  news_df['JobTitle'] = news_df['JobTitle'].str.replace('–','-')
  news_df['Link'] = news_df['Link'].apply(lambda x: x.split('&sjdu=')[0])
  news_df['Role'] = role

  indeed_data = indeed_data.append(news_df)

In [ ]:
indeed_data.drop_duplicates(subset = ['JobTitle','Company','Location', 'Role'], inplace=True)
indeed_data.to_csv('Indeed_incr_'+str(dt.today().date())+'.csv')
del indeed_data

In [ ]:
df = pd.read_csv('Indeed_incr_'+str(dt.today().date())+'.csv').iloc[:,1:]

#### Fetch job description

In [ ]:
def check_exception_skills(skills):
  res = []
  for x in ['C','R','Excel']:
    if f' {x}<' in skills or f' {x},' in skills or f' {x}.' in skills or f'>{x} ' in skills or f'.{x} ' in skills or f' {x} ' in skills:
      res.append(True)
    else:
      res.append(False)
  return res 

In [ ]:
skills_list = []
majors_list = []
degree_list = []
contents = []

for url in tqdm_notebook(df['Link']):
    try:
        data = requests.get(url)
        soup = BeautifulSoup(data.content, 'html.parser')
        content = str(soup.find_all('div', class_=["jobsearch-jobDescriptionText"]))
        skills = {v:True if k in content.lower() else False for k,v in skillz.items()}
        major = {v:True if k in content.lower() else False for k,v in majors.items()}
        degree = {v:True if k in content.lower() else False for k,v in degrees.items()}
        contents.append(content)
    #     skills_list.append({**skills,**major,**degree})
        
    except:
        skills = {k:'NA' for k in skillz.keys()}
        major = {k:'NA' for k in majors.keys()}
        degree = {k:'NA' for k in degrees.keys()}
        contents.append('')
    skills_list.append(skills)
    majors_list.append(major)
    degree_list.append(degree)
degree_df = pd.DataFrame(degree_list)
degree_df['No degree specified'] = ~degree_df.any(axis = 1)
temp = pd.concat([pd.DataFrame(skills_list),pd.DataFrame(majors_list),degree_df],axis = 1)
temp['content'] = contents
temp[['C','R','Excel']] = temp['content'].apply(lambda x: pd.Series(check_exception_skills(x)))
del temp['content']

In [ ]:
final = pd.concat([df,temp], axis = 1)
final = final.drop_duplicates(['JobTitle','Role','Company','Link'])

In [ ]:
final.to_csv('Indeed_Jobs_w-skills_'+str(dt.today().date())+'.csv')

### GlassDoor

In [ ]:
headers = {
    'user-agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36",
    'content-type': "application/json",
    'accept': "application/json"
}

In [ ]:
def extract(html):
    job_postings = []
    try:
        soup = BeautifulSoup(html, 'html.parser')

        job_postings = [{'JobTitle': [tag for tag in title][1].text.strip(),
                          'Company': company.div.text.strip(),
                          'Location': location.text.strip(),
                          'DatePosted':time_.text.strip(),
                          'Salary': [tag.text.split(' \xa0')[0].strip() for tag in title  if '$' in tag.text],
                          'Link':'https://www.glassdoor.com'+title.find('a',href = True)['href']
                         }

                            for title, company, location,time_ in 
                             zip(soup.find_all('div', 
                                               class_=["jobContainer"]),
                                 soup.find_all('div', 
                                               class_=["jobHeader"]),
                                soup.find_all('span', 
                                               class_=["subtle loc"]),
                                soup.find_all('span', 
                                               class_=["minor"]))
                        ]
    except Exception as e:
        print(e, job_postings)
        pass
    return job_postings

In [ ]:
jobs = pd.DataFrame()
for role in tqdm_notebook(roles):
#     print(role)
#     base_url = 'https://www.glassdoor.com/Job/{}-jobs-SRCH_KO0,18_IP{}.htm?jobType=fulltime'
    keyword = role.replace(' ','+')
    base_url = f'https://www.glassdoor.com/Job/jobs.htm?typedKeyword={keyword}&sc.keyword={keyword}&jobType=fulltime&sortBy=date_desc'
    s = requests.session()
    data = s.get(base_url, headers = headers)
    soup = BeautifulSoup(data.content, 'html.parser')
    next_url = 'https://www.glassdoor.com'+soup.find('li', class_="next").find('a',href = True)['href']
    pages = soup.find('div',class_ = 'cell middle hideMob padVertSm').text.strip().split(' ')[-1]
    max_pages = int(pages)
    
    final =  pd.DataFrame(columns = ['Company','JobTitle','Location','Salary','DatePosted','Link'])

    pages = min(1000,max_pages)

    for _ in range(1,pages):

        data = s.get(next_url, headers = headers)
        soup = BeautifulSoup(data.content, 'html.parser')
        extracted = extract(data.content)

        df = pd.DataFrame(extracted)
        final = final.append(df)
        try:
            next_url = 'https://www.glassdoor.com'+soup.find('li', class_="next").find('a',href = True)['href']
        except:
            break
        
    final['Salary'] = final['Salary'].apply(lambda x: x[0].split('(')[0].strip() if len(x)!=0 else 'NA')
    final = final.reset_index(drop = True)
    
    final['Role'] = role
    jobs = jobs.append(final)

In [ ]:
print(len(jobs),'\n',len(jobs.drop_duplicates(['Company','JobTitle','Role'])))

In [ ]:
jobs = jobs.drop_duplicates(['Company','JobTitle','Role'])

In [ ]:
jobs.to_csv('Glassdoor_incr_'+str(dt.today().date())+'.csv')

## Data Cleaning

### Indeed

In [ ]:
indeed = pd.read_csv('Indeed_Jobs_w-skills_'+str(dt.today().date())+'.csv').iloc[:,1:]

In [ ]:
indeed.head()

#### Extract Lat/Long and City, state

In [ ]:
indeed['Lat/Long'] = indeed['Location'].progress_apply(geocode)
indeed[['Lat','Long']] = pd.DataFrame(indeed['Lat/Long'].apply(pd.Series))

In [ ]:
def get_city_state_indeed(latlong):
    try:
#         location = geolocator.geocode(row['City'])
#         print(location.address.strip())
#         return location.address.split(',')[-3].strip()
        '''Using package uszipcode'''
        zipcode = search.by_coordinates(*latlong)[0]
        return zipcode.major_city, zipcode.state
    except:
        return None, None

In [ ]:
indeed[['City','State']] = indeed['Lat/Long'].progress_apply(lambda x:pd.Series(get_city_state_indeed(x)))

#### Extract Date of posting(Indeed)


In [ ]:
def find_days_indeed(x):
    if 'days ago' in x:
      days = re.findall('\d+',x)[0]
    else:
      days = '0'
    return str(eval('(dt.now() - timedelta(days = {})).date()'.format(days)))#str(eval('dt.now() - timedelta({} = {})'.format(unit, number)))

indeed['Date'] = indeed['DatePosted'].progress_apply(find_days_indeed)

In [ ]:
indeed['Salary'] = None
indeed = indeed[['Role', 'Company', 'JobTitle', 'DatePosted', 'Date','City', 'State', 'Link','Lat/Long', 'Location', 'Lat', 'Long', 
       'Excel', 'Python', 'R', 'SQL', 'C', 'Java', 'C++', 'Spark', 'Hadoop',
       'AWS', 'SAS', 'Matlab', 'Scala', 'Tableau', 'Hive', 'Tensorflow',
       'Javascript', 'Perl', 'Linux', 'Unix', 'Mapreduce', 'Scikit', 'HTML',
       'Oracle', 'Mongodb', 'Computer Science', 'Statistics', 'Mathematics',
       'Economics', 'Biology', 'Quantitative', 'Data Science', 'Analytics',
       'Chemical Engineer', 'High School', 'Bachelor', 'Master', 'PhD',
       'No degree specified']]

In [ ]:
indeed.to_csv('Indeed_Jobs_'+str(dt.today().date())+'.csv')

### Glassdoor

In [ ]:
glassdoor = pd.read_csv('Glassdoor_incr_'+str(dt.today().date())+'.csv')

#### Find skills and qualifications listed from Job Description

In [ ]:
skills_list = []
majors_list = []
degree_list = []
contents = []

for url in tqdm_notebook(glassdoor['Link']):
    try:
        for i in range(5):
          try:
            data = requests.get(url, headers = headers)
            break
          except ConnectionResetError:
            if i==4:
              assert False
            time.sleep(2)
            
        soup = BeautifulSoup(data.content, 'html.parser')
        content = soup.find_all('div', attrs={"id":"JobDescriptionContainer"})[0].text
        skills = {v:(True if k in content.lower() else False) for k,v in skillz.items()}
        major = {v:(True if k in content.lower() else False) for k,v in majors.items()}
        degree = {v:(True if k in content.lower() else False) for k,v in degrees.items()}
    #     skills_list.append({**skills,**major,**degree})
        contents.append(content)
        
    except Exception as e:
        skills = {k:'NA' for k in skillz.values()}
        major = {k:'NA' for k in majors.values()}
        degree = {k:'NA' for k in degrees.values()}
        contents.append('')
    skills_list.append(skills)
    majors_list.append(major)
    degree_list.append(degree)
degree_df = pd.DataFrame(degree_list)
degree_df['No degree specified'] = ~degree_df.any(axis = 1)
temp = pd.concat([pd.DataFrame(skills_list),pd.DataFrame(majors_list),degree_df],axis = 1)
temp = temp.replace('NA',False)
temp['content'] = contents
temp[['C','R','Excel']] = temp['content'].apply(lambda x: pd.Series(check_exception_skills(x)))

In [ ]:
glassdoor = pd.concat([glassdoor, temp], axis = 1)

In [ ]:
glassdoor = glassdoor.reset_index(drop = True)

In [ ]:
def get_city_state_glassdoor(row):
  x = row['Location']
  if ',' in x:
    return x.split(',')[0].strip(),x.split(',')[-1].strip()
  else:
    try:
      zipcode = search.by_coordinates(*row['Lat/Long'])[0]
      return zipcode.major_city, zipcode.state
    except:
      return None, None

def find_date_glassdoor(x):
  if 'hr' in x:
    days = '1'
  elif '+' in x:
    days = '40'
  elif 'd' in x:
    days = re.findall('\d+',x)[0]
  return str(eval('(dt.now() - timedelta(days = {})).date()'.format(days)))

In [ ]:
glassdoor['Lat/Long'] = glassdoor['Location'].progress_apply(geocode)
glassdoor[['Lat','Long']] = pd.DataFrame(glassdoor['Lat/Long'].apply(pd.Series))
glassdoor[['City','State']] = glassdoor.apply(lambda x: pd.Series(get_city_state_glassdoor(x)),axis = 1)
glassdoor['Date'] = glassdoor['DatePosted'].apply(find_date_glassdoor)

In [ ]:
glassdoor = glassdoor[['Role', 'Company', 'JobTitle', 'DatePosted', 'Date','City', 'State', 'Link','Lat/Long', 'Location', 'Lat', 'Long', 'Salary',
       'Excel', 'Python', 'R', 'SQL', 'C', 'Java', 'C++', 'Spark', 'Hadoop',
       'AWS', 'SAS', 'Matlab', 'Scala', 'Tableau', 'Hive', 'Tensorflow',
       'Javascript', 'Perl', 'Linux', 'Unix', 'Mapreduce', 'Scikit', 'HTML',
       'Oracle', 'Mongodb', 'Computer Science', 'Statistics', 'Mathematics',
       'Economics', 'Biology', 'Quantitative', 'Data Science', 'Analytics',
       'Chemical Engineer', 'High School', 'Bachelor', 'Master', 'PhD',
       'No degree specified']]

In [ ]:
glassdoor.to_csv('Glassdoor_Jobs_'+str(dt.today().date())+'.csv')

## Combine Datasets

In [ ]:
glassdoor['Source'] = 'Glassdoor'
indeed['Source'] = 'Indeed'
latest_jobs = glassdoor.append(indeed)
latest_jobs.to_csv('Jobs_I+G_'+str(dt.today().date())+'.csv')

#### Get Industry info

In [ ]:
companies = pd.read_csv('Companies.csv').iloc[:,1:]
latest_jobs = pd.read_csv('Jobs_I+G_'+str(dt.today().date())+'.csv')
# latest_jobs = pd.read_csv('Jobs_I+G_2020-04-26.csv').iloc[:,1:]

In [ ]:
allcompanies = companies[['company']].drop_duplicates()['company'].str.lower().values.tolist()

In [ ]:
def fetch_best_match(company):
  if company.lower() in allcompanies:
    best = company
    return best
  else:
    return process.extractOne(company,companies['company'].tolist(),scorer=fuzz.token_set_ratio)[0]

In [ ]:
latest_jobs['match']= latest_jobs['Company'].progress_apply(lambda x: pd.Series(fetch_best_match(x)))

In [ ]:
final = latest_jobs.merge(companies, left_on = 'match',right_on = 'company',how = 'left')[latest_jobs.columns.tolist()+['category']]
del final['match']
final.rename(columns = {'category':'Industry'},inplace = True)
final.to_csv('Jobs_Indeed+Glassdoor_'+str(dt.today().date())+'.csv')